In [8]:
%load_ext autoreload
%autoreload 2
import torch
import dgl
import malt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
net = malt.models.supervised_model.SimpleSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.NeuralNetworkRegressor(
        in_features=128, out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)


In [14]:
policy = malt.policy.Greedy(
    utility_function=malt.policy.utility_functions.expected_improvement,
)

trainer = malt.trainer.get_default_trainer(n_epochs=10)
center = malt.agents.center.NaiveCenter(name="NaiveCenter")
vendor, assayer = malt.fake_tasks.collections.esol()

player = malt.agents.player.AutonomousPlayer(
    name="AutonomousPlayer",
    center=center,
    policy=policy,
    model=net,
    trainer=trainer,
)

center.register(vendor)
center.register(assayer)
center.register(player)

catalogue = vendor.catalogue()()

while len(player.portfolio) < len(catalogue):
    points_to_acquire = player.prioritize(catalogue-player.portfolio)
    query_receipt = player.query(
        points_to_acquire,
        vendor=vendor,
        assayers=[assayer],
    )
    assert query_receipt is not None
    quote = player.check(query_receipt)
    order_receipt = player.order(quote)
    assert order_receipt is not None
    report = player.check(order_receipt)[0]
    player.portfolio += report.points
    player.train()

print([len(point.smiles) for point in player.portfolio])

Processing dgl graphs from scratch...
Processing molecule 1000/4200
Processing molecule 2000/4200
Processing molecule 3000/4200
Processing molecule 4000/4200


KeyboardInterrupt: 